# Data import

To start the database used in this notebook, please run `docker run -d --name pgData -p 5432:5432 postgres:latest`

To stop the database (and remove it), run `docker stop pgData && docker rm pgData`

To connect and get a terminal to the postgres docker, run `docker exec -it pbdw2018_hackathon-master_data_postgres_1 /bin/bash`

You can then do things like

dropdb mydata -U postgres

or

psql postgres -U postgres


In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import csv

# Connect to an existing database
try:
    conn = psycopg2.connect("host=postgresdb user=postgres password=postgres")
except:
    print ("Error:  unable to connect to the database")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command to end all connections to the db
try:
    cur.execute("SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = 'mydata' AND pid <> pg_backend_pid();")
except:
    print("Error killing database connections, perhaps it does not exist?")
    
# Execute a command to drop the table
try:
    cur.execute("DROP DATABASE mydata")
except:
    print("Error while dropping database, perhaps it does not exist?")

# Execute a command to create a new table
try:
    cur.execute("CREATE DATABASE mydata;")
except:
    print("Error while creating database, does it already exist?")
# Close database connection
conn.close()

Error while dropping database, perhaps it does not exist?


In [2]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
# Open a cursor to perform database operations
cur = conn.cursor()
# Execute a command: this creates a new table
try:
    cur.execute("""CREATE TABLE PBDWHackathon2018 (
    ID text PRIMARY KEY,
    PatientID text,
    Age text,
    Clinical_T_Stage text,
    Clinical_N_Stage text,
    Clinical_M_Stage text,
    Overall_Stage text,
    Histology text,
    Gender text,
    SurvivalTime text,
    DeadStatus text
    );
    """)
except:
    print("Error while creating table, does it already exist?")
# Close database connection
conn.close()

In [3]:
import pandas
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

#open CSV file
df = pandas.read_csv('Lung1Clinical.csv')
for index, row in df.iterrows():
    #print(str(index) + " | " + row.Idfu)
    #cur.execute
    cur.execute("""INSERT INTO PBDWHackathon2018 (ID, PatientID, Age, Clinical_T_Stage, Clinical_N_Stage, Clinical_M_Stage, Overall_Stage, Histology, Gender, SurvivalTime, DeadStatus) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
          (index, row.PatientID, row.age, row["clinical.T.Stage"], row["Clinical.N.Stage"], row["Clinical.M.Stage"], row["Overall.Stage"], row.Histology, row.gender, row["Survival.time"], row["deadstatus.event"]))
conn.commit()

In [4]:
# Connect to an existing database
conn = psycopg2.connect("host=postgresdb user=postgres dbname=mydata password=postgres")
cur = conn.cursor()

cur.execute("""SELECT *
    FROM PBDWHackathon2018;""")

results = cur.fetchall()
conn.close()

for row in results:
    print(row)

('0', 'LUNG1-001', '78.7515', '2', '3', '0', 'IIIb', 'large cell', 'male', '2165', '1')
('1', 'LUNG1-002', '83.8001', '2', '0', '0', 'I', 'squamous cell carcinoma', 'male', '155', '1')
('2', 'LUNG1-003', '68.1807', '2', '3', '0', 'IIIb', 'large cell', 'male', '256', '1')
('3', 'LUNG1-004', '70.8802', '2', '1', '0', 'II', 'squamous cell carcinoma', 'male', '141', '1')
('4', 'LUNG1-005', '80.4819', '4', '2', '0', 'IIIb', 'squamous cell carcinoma', 'male', '353', '1')
('5', 'LUNG1-006', '73.8864', '3', '1', '0', 'IIIa', 'squamous cell carcinoma', 'male', '173', '1')
('6', 'LUNG1-007', '81.5288', '2', '2', '0', 'IIIa', 'squamous cell carcinoma', 'male', '137', '1')
('7', 'LUNG1-008', '71.666', '2', '2', '0', 'IIIa', 'adenocarcinoma', 'male', '77', '1')
('8', 'LUNG1-009', '56.1342', '2', '2', '0', 'IIIa', 'squamous cell carcinoma', 'male', '131', '1')
('9', 'LUNG1-010', '71.0554', '4', '3', '0', 'IIIb', 'squamous cell carcinoma', 'female', '2119', '0')
